In [1]:
# In this exercise you will train a CNN on the FULL Cats-v-dogs dataset
# This will require you doing a lot of data preprocessing because
# the dataset isn't split into training and validation for you
# This code block has all the required inputs
import os
import random
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copy

In [ ]:
# This code block downloads the full Cats-v-Dogs dataset and stores it as 
# cats-and-dogs.zip. It then unzips it to /tmp
# which will create a tmp/PetImages directory containing subdirectories
# called 'Cat' and 'Dog' (that's how the original researchers structured it)
# If the URL doesn't work, 
# .   visit https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765
# And right click on the 'Download Manually' link to get a new URL

# !wget --no-check-certificate \
#     "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip" \
#     -O "/tmp/cats-and-dogs.zip"

# local_zip = '/tmp/cats-and-dogs.zip'
# zip_ref = zipfile.ZipFile(local_zip, 'r')
# zip_ref.extractall('/tmp')
# zip_ref.close()

In [2]:
print(len(os.listdir('tmp/PetImages/Cat/')))
print(len(os.listdir('tmp/PetImages/Dog/')))

# Expected Output:
# 12501
# 12501

12501
12501


In [10]:
# Use os.mkdir to create your directories
# You will need a directory for cats-v-dogs, and subdirectories for training
# and testing. These in turn will need subdirectories for 'cats' and 'dogs'

# print('cats-v-dogs')
# os.mkdir('tmp/cats-v-dogs/')
# modes = ['training', 'testing']
# labels = ['dogs', 'cats']

# for mode in modes:
#     print(f'tmp/cats-v-dogs/{mode}')
#     os.mkdir(f'tmp/cats-v-dogs/{mode}')
#     for label in labels:
#         print(f'tmp/cats-v-dogs/{mode}/{label}')
#         os.mkdir('tmp/cats-v-dogs/{}/{}/'.format(mode, label))

cats-v-dogs
tmp/cats-v-dogs/training
tmp/cats-v-dogs/training/dogs
tmp/cats-v-dogs/training/cats
tmp/cats-v-dogs/testing
tmp/cats-v-dogs/testing/dogs
tmp/cats-v-dogs/testing/cats


In [3]:
# Write a python function called split_data which takes
# a SOURCE directory containing the files
# a TRAINING directory that a portion of the files will be copied to
# a TESTING directory that a portion of the files will be copie to
# a SPLIT SIZE to determine the portion
# The files should also be randomized, so that the training set is a random
# X% of the files, and the test set is the remaining files
# SO, for example, if SOURCE is PetImages/Cat, and SPLIT SIZE is .9
# Then 90% of the images in PetImages/Cat will be copied to the TRAINING dir
# and 10% of the images will be copied to the TESTING dir
# Also -- All images should be checked, and if they have a zero file length,
# they will not be copied over
#
# os.listdir(DIRECTORY) gives you a listing of the contents of that directory
# os.path.getsize(PATH) gives you the size of the file
# copyfile(source, destination) copies a file from source to destination
# random.sample(list, len(list)) shuffles a list
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    # List of images
    data = os.listdir(SOURCE)
    
    # Check and remove files with zero length
    for img in data:
        if os.path.getsize(SOURCE + img) == 0:
            print('Remove', img)
            data.remove(img)
            
    # Randomise list of images
    data = random.sample(data, len(data))
    
    # Split
    train_size = int(len(data) * SPLIT_SIZE)
    train_set = data[:train_size]
    test_set = data[train_size:]
    
    # Copy to directories
    subsets = [train_set, test_set]
    dests = [TRAINING, TESTING]
    for subset, dest in zip(subsets, dests):
        print('\n{}'.format(dest.split('/')[-3].upper()))
        for i, img in enumerate(subset):
            copy(SOURCE + img, dest)
            if i % 100 == 0:
                print(i, end=' ')
        

CAT_SOURCE_DIR = "tmp/PetImages/Cat/"
TRAINING_CATS_DIR = "tmp/cats-v-dogs/training/cats/"
TESTING_CATS_DIR = "tmp/cats-v-dogs/testing/cats/"
DOG_SOURCE_DIR = "tmp/PetImages/Dog/"
TRAINING_DOGS_DIR = "tmp/cats-v-dogs/training/dogs/"
TESTING_DOGS_DIR = "tmp/cats-v-dogs/testing/dogs/"

split_size = .9
# print('\n---CAT')
# split_data(CAT_SOURCE_DIR, TRAINING_CATS_DIR, TESTING_CATS_DIR, split_size)
# print('\n\n---DOG')
# split_data(DOG_SOURCE_DIR, TRAINING_DOGS_DIR, TESTING_DOGS_DIR, split_size)

# Expected output
# 666.jpg is zero length, so ignoring
# 11702.jpg is zero length, so ignoring

In [4]:
print(len(os.listdir('tmp/cats-v-dogs/training/cats/')))
print(len(os.listdir('tmp/cats-v-dogs/training/dogs/')))
print(len(os.listdir('tmp/cats-v-dogs/testing/cats/')))
print(len(os.listdir('tmp/cats-v-dogs/testing/dogs/')))

# Expected output:
# 11250
# 11250
# 1250
# 1250

11977
11250
1250
1250


In [12]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense

# DEFINE A KERAS MODEL TO CLASSIFY CATS V DOGS
# USE AT LEAST 3 CONVOLUTION LAYERS
model = tf.keras.models.Sequential([
    Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPool2D(pool_size=2, strides=2),
    Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=2, strides=2),
    Flatten(),
    Dense(units=512, activation='relu'),
    Dense(units=1, activation='sigmoid')
])

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

In [13]:
TRAINING_DIR = 'tmp/cats-v-dogs/training/'
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(150, 150),
    class_mode='binary',
    batch_size=16
)

VALIDATION_DIR = 'tmp/cats-v-dogs/testing/'
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(150, 150),
    class_mode='binary',
    batch_size=16
)


# Expected Output:
# Found 22498 images belonging to 2 classes.
# Found 2500 images belonging to 2 classes.

Found 23213 images belonging to 2 classes.
Found 2499 images belonging to 2 classes.


In [14]:
import warnings
warnings.filterwarnings('ignore')

history = model.fit_generator(train_generator,
                              epochs=15,
                              verbose=1,
                              validation_data=validation_generator)

# The expectation here is that the model will train, and that accuracy will be > 95% on both training and validation
# i.e. acc:A1 and val_acc:A2 will be visible, and both A1 and A2 will be > .9

Epoch 1/15
1451/1451 [==============================] - 297s 205ms/step - loss: 0.8940 - acc: 0.6832 - val_loss: 0.5601 - val_acc: 0.7175
Epoch 2/15
1451/1451 [==============================] - 327s 225ms/step - loss: 0.4622 - acc: 0.7895 - val_loss: 0.3834 - val_acc: 0.8319
Epoch 3/15
1451/1451 [==============================] - 318s 219ms/step - loss: 0.4090 - acc: 0.8196 - val_loss: 0.4279 - val_acc: 0.8243
Epoch 4/15
1451/1451 [==============================] - 314s 216ms/step - loss: 0.3729 - acc: 0.8420 - val_loss: 0.3519 - val_acc: 0.8519
Epoch 5/15
1451/1451 [==============================] - 310s 214ms/step - loss: 0.3484 - acc: 0.8537 - val_loss: 0.3526 - val_acc: 0.8639
Epoch 6/15
1451/1451 [==============================] - 295s 203ms/step - loss: 0.3424 - acc: 0.8629 - val_loss: 0.3598 - val_acc: 0.8679
Epoch 7/15
1451/1451 [==============================] - 286s 197ms/step - loss: 0.3224 - acc: 0.8714 - val_loss: 0.3185 - val_acc: 0.8747
Epoch 8/15
1451/1451 [============

KeyboardInterrupt: 

In [15]:
# PLOT LOSS AND ACCURACY
%matplotlib inline

import matplotlib.image  as mpimg
import matplotlib.pyplot as plt

#-----------------------------------------------------------
# Retrieve a list of list results on training and test data
# sets for each training epoch
#-----------------------------------------------------------
acc=history.history['acc']
val_acc=history.history['val_acc']
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(acc)) # Get number of epochs

#------------------------------------------------
# Plot training and validation accuracy per epoch
#------------------------------------------------
plt.plot(epochs, acc, 'r', "Training Accuracy")
plt.plot(epochs, val_acc, 'b', "Validation Accuracy")
plt.title('Training and validation accuracy')
plt.figure()

#------------------------------------------------
# Plot training and validation loss per epoch
#------------------------------------------------
plt.plot(epochs, loss, 'r', "Training Loss")
plt.plot(epochs, val_loss, 'b', "Validation Loss")


plt.title('Training and validation loss')

# Desired output. Charts with training and validation metrics. No crash :)

NameError: name 'history' is not defined